## Reproduce Ershadi (2021)

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Effective medium model imports
from effmed.lib.matrix_model import effective_medium
from effmed.lib.supplemental import dB, rotational_transform, coherence, phase_gradient2d

## Figure 3

In [ ]:
# Initialize the model at a chosen center frequency
em = effective_medium()
em.system_setup(fc=300e6)

# Set the material properties
lams = [.5,0.,0.5]

# Model domain (4000 m depth)
zs = np.linspace(.01,4000,4000)

# COF orientation relative to the instrument axes
psis = np.array([45,45,45,45,135,135,120, 120])*np.pi/180. + .0001
thetas = np.zeros(len(psis))
layer_dzs = 500*np.ones(len(psis))

# COF eigenvalues for all layers
dlams = np.array([.025, 0.2, 0.2, 0.2, 0.2, 0.45, 0.2, 0.2])
lams = np.empty((8,3))
for i in range(8):
    lams[i] = np.array([(.5-dlams[i])/2.,(.5+dlams[i])/2.,0.5])

em.ice_properties(idctx='biaxial',epsr=3.15,theta=thetas[0],psi=psis[0],lam=lams[0])
    
# Anisotropic scattering for all layers
gammaxs = 1e-12*np.ones_like(zs)
gammays = 1e-12*np.ones_like(zs)
gammays[zs>1000.] = 1e-11
gammays[zs>1500.] = 1e-13
gammays[zs>2500.] = 1e-14
gammays[zs>3000.] = 1e-12
gammas = np.transpose([gammaxs,gammays])

# Rotation of the scattering interface
psi_gammas = np.pi/4.*np.ones(len(zs))
psi_gammas[zs>2000.] = 3.*np.pi/4.
psi_gammas[zs>3000.] = 120*np.pi/180.

# Solve model for all layers
em.solve(zs,layer_dzs,thetas,psis,lams,gammas=gammas,psi_gammas=psi_gammas)

# Rotate model result to all azimuths for a 2-d image
rotational_transform(em,n_thetas=200)
Θs,Ds = np.meshgrid(em.thetas,em.range)

# Co-polarized phase coherence and gradient from the model result
coherence(em)
phase_gradient2d(em)

In [ ]:
plt.figure(figsize=(12, 9))

plt.subplot(141)
HH = np.real(dB(em.HH**2.))
HH = np.transpose(HH) - np.mean(HH,axis=1)
HH = np.transpose(HH)
plt.contourf(Θs,Ds,HH,cmap='hot',levels=np.arange(-15,15,.1),vmin=-15,vmax=15,extend='both')
plt.colorbar(label='Co-Polarized',location='top',ticks=np.arange(-15,16,5))
plt.gca().invert_yaxis()
plt.xlabel('Azimuth')
plt.ylabel('Depth (m)')
plt.xticks([0,np.pi/2.,np.pi])
plt.gca().set_xticklabels(['0','π/2','π'])

plt.subplot(143)
plt.tick_params(labelleft=False)
HV = np.real(dB(em.HV**2.))
HV = np.transpose(HV) - np.mean(HV,axis=1)
HV = np.transpose(HV)
plt.contourf(Θs,Ds,HV,cmap='hot',levels=np.arange(-15,15,.1),vmin=-15,vmax=15,extend='both')
plt.colorbar(label='Cross-Polarized',location='top',ticks=np.arange(-15,16,5))
plt.gca().invert_yaxis()
plt.xlabel('Azimuth (rad)')
plt.xticks([0,np.pi/2.,np.pi])
plt.gca().set_xticklabels(['0','π/2','π'])

plt.subplot(142)
plt.tick_params(labelleft=False)
plt.contourf(Θs,Ds,np.angle(em.chhvv),cmap='twilight_shifted',levels=100,vmin=-np.pi,vmax=np.pi)
cbar = plt.colorbar(label='$ϕ_{hhvv}$',location='top',ticks=np.array([-np.pi,0,np.pi]))
cbar.set_ticklabels(['-π','0','π'])
plt.gca().invert_yaxis()
plt.xlabel('Azimuth')
plt.xticks([0,np.pi/2.,np.pi])
plt.gca().set_xticklabels(['0','π/2','π'])

dϕ = em.dphi_dz.copy()
clim = .05
dϕ[dϕ>clim]=clim
dϕ[dϕ<-clim]=-clim

plt.subplot(144)
plt.tick_params(labelleft=False)
plt.contourf(Θs,Ds,dϕ,cmap='seismic',levels=100,vmin=-clim,vmax=clim,extend='both')
plt.colorbar(label='$\partial ϕ/\partial z$',location='top',ticks=[-clim,0,clim])
plt.gca().invert_yaxis()
plt.xlabel('Azimuth')
plt.xticks([0,np.pi/2.,np.pi])
plt.gca().set_xticklabels(['0','π/2','π'])

plt.tight_layout()